<a href="https://colab.research.google.com/github/Stravanni/Basi_di_dati/blob/main/01_SQL_sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduzione a SQL
@author: giovanni.simonini@unimore.it

## To run: 
- to run a cell: SHIFT + ENTER
  
## Schema Università

S (<u>Matr</u>,SNome,Citta,ACorso)

D(<u>CD</u>,CNome,Citta)

C(<u>CC</u>,CNome,CD)
- FOREIGN KEY (CD) REFERENCES D(CD)

E(<u>Matr,CC</u>,DATA,VOTO)
- FOREIGN KEY (Matr) REFERENCES S(Matr)
- FOREIGN KEY (CC) REFERENCES C(CC)
 

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [5]:
engine = create_engine('sqlite://', echo=False)

## Create the the tables

In [6]:
q = '''
CREATE TABLE S (
    Matr VARCHAR(45),
    SNome VARCHAR(45),
    Citta VARCHAR(45),
    ACorso INT,
    PRIMARY KEY (Matr)
);
'''
engine.execute(q)

q = '''
CREATE TABLE D(
 CD VARCHAR(45),
 CNome VARCHAR(45),
 Citta VARCHAR(45),
 PRIMARY KEY (CD)
);
'''
engine.execute(q)

q = '''
CREATE TABLE C(
 CC VARCHAR(45),
 CNome VARCHAR(45),
 CD VARCHAR(45),
 PRIMARY KEY (CC),
 FOREIGN KEY (CD) REFERENCES D(CD)
);
'''
engine.execute(q)

q = '''
CREATE TABLE E(
 Matr VARCHAR(45),
 CC VARCHAR(45),
 DATA DATE,
 VOTO INT,
 PRIMARY KEY (Matr, CC),
 FOREIGN KEY (Matr) REFERENCES S(Matr),
 FOREIGN KEY (CC) REFERENCES C(CC)
);
'''
engine.execute(q)


q = '''
INSERT INTO S (Matr, SNome, Citta, ACorso)
VALUES
('M1','Lucia Quaranta','SA',1),
('M2','Giacomo Tedesco','PA',2),
('M3','Carla Longo','MO',1),
('M4','Ugo Rossi','MO',1),
('M5','Valeria Neri','MO',2),
('M6','Giuseppe Verdi','BO',1),
('M7','Maria Rossi',null,1);
'''
engine.execute(q)


q = '''
INSERT INTO D (CD, CNome, Citta)
VALUES
('D1','Paolo Rossi','MO'),
('D2','Maria Pastore','BO'),
('D3','Paola Caboni','FI');
'''
engine.execute(q)


q = '''
INSERT INTO C (CC,CNome, CD)
VALUES
('C1','Fisica 1','D1'),
('C2','Analisi Matematica 1','D2'),
('C3','Fisica 2','D1'),
('C4','Analisi Matematica 2','D3');
'''
engine.execute(q)


q = '''
INSERT INTO E (Matr,CC,Data,Voto)
VALUES
('M1','C1','1995-06-29',24),
('M1','C2','1996-08-09',33),
('M1','C3','1996-03-12',30),
('M2','C1','1995-06-29',28),
('M2','C2','1996-07-07',24),
('M3','C2','1996-07-07',27),
('M3','C3','1996-11-11',25),
('M4','C3','1996-11-11',33),
('M6','C2','1996-01-02',28),
('M7','C1','1995-06-29',24),
('M7','C2','1996-04-11',26),
('M7','C3','1996-06-23',27);
'''
engine.execute(q)

## Query the DB

In [7]:
# TEMPLATE QUERY
q = '''
SELECT *
FROM S,C,D,E
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso,CC,CNome,CD,CD,CNome,Citta,Matr,CC,DATA,VOTO
0,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M1,C1,1995-06-29,24
1,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M1,C2,1996-08-09,33
2,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M1,C3,1996-03-12,30
3,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M2,C1,1995-06-29,28
4,M1,Lucia Quaranta,SA,1,C1,Fisica 1,D1,D1,Paolo Rossi,MO,M2,C2,1996-07-07,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,M7,Maria Rossi,None,1,C4,Analisi Matematica 2,D3,D3,Paola Caboni,FI,M4,C3,1996-11-11,33
1004,M7,Maria Rossi,None,1,C4,Analisi Matematica 2,D3,D3,Paola Caboni,FI,M6,C2,1996-01-02,28
1005,M7,Maria Rossi,None,1,C4,Analisi Matematica 2,D3,D3,Paola Caboni,FI,M7,C1,1995-06-29,24
1006,M7,Maria Rossi,None,1,C4,Analisi Matematica 2,D3,D3,Paola Caboni,FI,M7,C2,1996-04-11,26


In [8]:

# Operatori relazionali : <attr> <op-rel> <cost> 
# dove <op-rel> ∈ {=, <>, >, >=, <, <=}
# 
# Studenti del secondo anno di corso
q = ''' 
SELECT *
FROM S
WHERE ACorso=2;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso
0,M2,Giacomo Tedesco,PA,2
1,M5,Valeria Neri,MO,2


In [9]:
# Esami con voto compreso tra 24 e 28
q = '''
SELECT *
FROM E
WHERE Voto > 24
AND Voto <= 28;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,CC,DATA,VOTO
0,M2,C1,1995-06-29,28
1,M3,C2,1996-07-07,27
2,M3,C3,1996-11-11,25
3,M6,C2,1996-01-02,28
4,M7,C2,1996-04-11,26
5,M7,C3,1996-06-23,27


In [10]:
# Operatore di set : <attr> IN (<cost1>, ..., <costN>)
#
# Esami con voto pari a 29, 30 oppure con lode (voto pari a 33)
q = '''
SELECT *
FROM E
WHERE Voto IN (29,30,33);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,CC,DATA,VOTO
0,M1,C2,1996-08-09,33
1,M1,C3,1996-03-12,30
2,M4,C3,1996-11-11,33


In [11]:
# Operatore di confronto stringhe : <attr> LIKE <stringa> 
# dove <stringa> puo` contenere i caratteri speciali   (carattere arbitrario) e % (stringa arbitraria)
#
# Studenti il cui nome inizia con A e termina con O

q = '''
SELECT *
FROM S
WHERE SNome LIKE 'V%i'
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso
0,M5,Valeria Neri,MO,2


In [12]:
# Ordinamento del risultato: ORDER BY <attr> [ASC|DESC]
#
# Studenti di Modena ordinati in senso ascendente rispetto all’anno di corso
q = '''
SELECT Matr,ACorso 
FROM S
WHERE Citta = 'MO' 
ORDER BY ACorso DESC
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,ACorso
0,M5,2
1,M3,1
2,M4,1


In [13]:
# L’ordinamento deve essere fatto rispetto a uno o piu' elementi della <lista-select>: 
# un tale elemento puo' essere indicato anche riportando la sua posizione nella <lista-select>.
#
# Esami del corso C1 ordinati in senso discendente rispetto al voto espresso in sessantesimi, 
# e a parita` di voto rispetto alla matricola

q = '''
SELECT Matr,CC,(60*Voto)/30 
FROM E
WHERE CC='C1'
ORDER BY 3 DESC, Matr DESC
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,CC,(60*Voto)/30
0,M2,C1,56
1,M7,C1,48
2,M1,C1,48


## JOIN operators

In [14]:
# Patriamo dal prodotto cartesiano: 
# Cioe' la combinazione di tutte le possibili tuple delle tabelle elencate
# nella clausola FROM
q = '''
SELECT *
FROM S,D,E,C
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso,CD,CNome,Citta,Matr,CC,DATA,VOTO,CC,CNome,CD
0,M1,Lucia Quaranta,SA,1,D1,Paolo Rossi,MO,M1,C1,1995-06-29,24,C1,Fisica 1,D1
1,M1,Lucia Quaranta,SA,1,D1,Paolo Rossi,MO,M1,C1,1995-06-29,24,C2,Analisi Matematica 1,D2
2,M1,Lucia Quaranta,SA,1,D1,Paolo Rossi,MO,M1,C1,1995-06-29,24,C3,Fisica 2,D1
3,M1,Lucia Quaranta,SA,1,D1,Paolo Rossi,MO,M1,C1,1995-06-29,24,C4,Analisi Matematica 2,D3
4,M1,Lucia Quaranta,SA,1,D1,Paolo Rossi,MO,M1,C2,1996-08-09,33,C1,Fisica 1,D1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,M7,Maria Rossi,None,1,D3,Paola Caboni,FI,M7,C2,1996-04-11,26,C4,Analisi Matematica 2,D3
1004,M7,Maria Rossi,None,1,D3,Paola Caboni,FI,M7,C3,1996-06-23,27,C1,Fisica 1,D1
1005,M7,Maria Rossi,None,1,D3,Paola Caboni,FI,M7,C3,1996-06-23,27,C2,Analisi Matematica 1,D2
1006,M7,Maria Rossi,None,1,D3,Paola Caboni,FI,M7,C3,1996-06-23,27,C3,Fisica 2,D1


In [15]:
q = '''
SELECT * 
FROM S,E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso,Matr,CC,DATA,VOTO
0,M1,Lucia Quaranta,SA,1,M1,C1,1995-06-29,24
1,M1,Lucia Quaranta,SA,1,M1,C2,1996-08-09,33
2,M1,Lucia Quaranta,SA,1,M1,C3,1996-03-12,30
3,M1,Lucia Quaranta,SA,1,M2,C1,1995-06-29,28
4,M1,Lucia Quaranta,SA,1,M2,C2,1996-07-07,24
...,...,...,...,...,...,...,...,...
79,M7,Maria Rossi,None,1,M4,C3,1996-11-11,33
80,M7,Maria Rossi,None,1,M6,C2,1996-01-02,28
81,M7,Maria Rossi,None,1,M7,C1,1995-06-29,24
82,M7,Maria Rossi,None,1,M7,C2,1996-04-11,26


In [19]:
# Il Join, per definizione e' un "filtro" sul prodotto cartesiano
# cioe', in algebra relazionale, una selezione sul prodotto cartesiano
q = '''
SELECT *
FROM S,E
WHERE S.Matr=E.Matr
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# # che equivale a:
# q = '''
# SELECT *
# FROM S JOIN E on S.Matr=E.Matr
# '''
# res = engine.execute(q)
# df = pd.DataFrame(res.fetchall())
# df.columns = res.keys()
# df

# # e anche a:
# q = '''
# SELECT *
# FROM S natural JOIN E
# '''
# res = engine.execute(q)
# df = pd.DataFrame(res.fetchall())
# df.columns = res.keys()
# df

,Matr,SNome,Citta,ACorso,Matr,CC,DATA,VOTO
0,M1,Lucia Quaranta,SA,1,M1,C1,1995-06-29,24
1,M1,Lucia Quaranta,SA,1,M1,C2,1996-08-09,33
2,M1,Lucia Quaranta,SA,1,M1,C3,1996-03-12,30
3,M2,Giacomo Tedesco,PA,2,M2,C1,1995-06-29,28
4,M2,Giacomo Tedesco,PA,2,M2,C2,1996-07-07,24
5,M3,Carla Longo,MO,1,M3,C2,1996-07-07,27
6,M3,Carla Longo,MO,1,M3,C3,1996-11-11,25
7,M4,Ugo Rossi,MO,1,M4,C3,1996-11-11,33
8,M6,Giuseppe Verdi,BO,1,M6,C2,1996-01-02,28
9,M7,Maria Rossi,None,1,M7,C1,1995-06-29,24


In [20]:
# Pero' possono esseci studenti che non hanno sostenuto esami.
# Ad esempio:

q = '''
SELECT *
FROM S
WHERE S.Matr not in (SELECT Matr FROM E)
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso
0,M5,Valeria Neri,MO,2


In [21]:
# Se volessimo anche questi studenti nell'output del join
# dobbiamo usare un LEFT join

q = '''
SELECT * FROM S LEFT JOIN E on S.Matr=E.Matr;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso,Matr,CC,DATA,VOTO
0,M1,Lucia Quaranta,SA,1,M1,C1,1995-06-29,24.0
1,M1,Lucia Quaranta,SA,1,M1,C2,1996-08-09,33.0
2,M1,Lucia Quaranta,SA,1,M1,C3,1996-03-12,30.0
3,M2,Giacomo Tedesco,PA,2,M2,C1,1995-06-29,28.0
4,M2,Giacomo Tedesco,PA,2,M2,C2,1996-07-07,24.0
5,M3,Carla Longo,MO,1,M3,C2,1996-07-07,27.0
6,M3,Carla Longo,MO,1,M3,C3,1996-11-11,25.0
7,M4,Ugo Rossi,MO,1,M4,C3,1996-11-11,33.0
8,M5,Valeria Neri,MO,2,None,None,None,NaN
9,M6,Giuseppe Verdi,BO,1,M6,C2,1996-01-02,28.0


In [22]:
# E se volessi:
# Combinazioni di studenti e di docenti residenti nella stessa citta'
# inclusi gli studenti che risiedono in una citta' 
# che non ha corrispondenza nella relazione dei docenti
q = '''
SELECT *
FROM S JOIN D ON S.Citta=D.Citta;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso,CD,CNome,Citta
0,M3,Carla Longo,MO,1,D1,Paolo Rossi,MO
1,M4,Ugo Rossi,MO,1,D1,Paolo Rossi,MO
2,M5,Valeria Neri,MO,2,D1,Paolo Rossi,MO
3,M6,Giuseppe Verdi,BO,1,D2,Maria Pastore,BO


In [23]:
# con anche gli studenti di citta' da cui non viene nessun docente:
q = '''
SELECT *
FROM S LEFT JOIN D ON S.Citta=D.Citta;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso,CD,CNome,Citta
0,M1,Lucia Quaranta,SA,1,None,None,None
1,M2,Giacomo Tedesco,PA,2,None,None,None
2,M3,Carla Longo,MO,1,D1,Paolo Rossi,MO
3,M4,Ugo Rossi,MO,1,D1,Paolo Rossi,MO
4,M5,Valeria Neri,MO,2,D1,Paolo Rossi,MO
5,M6,Giuseppe Verdi,BO,1,D2,Maria Pastore,BO
6,M7,Maria Rossi,None,1,None,None,None


In [24]:
# Nella clausola FROM e' possibile esprimere anche piu' condizioni di join.
# 
# Ad esempio, se volessimo:
# Per ogni esame con voto superiore a 24 riportare il nome dello studente 
# e il codice del docente del corso
q = '''
SELECT S.SNome,C.CD, E.voto
FROM (S JOIN E ON (S.Matr=E.Matr))
		JOIN C ON (E.CC=C.CC)
WHERE Voto > 24;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome,CD,VOTO
0,Lucia Quaranta,D2,33
1,Lucia Quaranta,D1,30
2,Giacomo Tedesco,D1,28
3,Carla Longo,D2,27
4,Carla Longo,D1,25
5,Ugo Rossi,D1,33
6,Giuseppe Verdi,D2,28
7,Maria Rossi,D2,26
8,Maria Rossi,D1,27


In [25]:
# che equivale a:
q = '''
SELECT S.SNome,C.CD, E.voto
FROM S,E,C
WHERE S.Matr=E.Matr 
AND E.CC=C.CC
AND Voto > 24;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome,CD,VOTO
0,Lucia Quaranta,D2,33
1,Lucia Quaranta,D1,30
2,Giacomo Tedesco,D1,28
3,Carla Longo,D2,27
4,Carla Longo,D1,25
5,Ugo Rossi,D1,33
6,Giuseppe Verdi,D2,28
7,Maria Rossi,D2,26
8,Maria Rossi,D1,27


In [26]:
# Con gli outer join, certi attributi vengono duplicati (per definizione)
# Ad sempio:
q = '''
SELECT S.Matr,S.Citta,D.CD, D.Citta
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,Citta,CD,Citta
0,M1,SA,None,None
1,M2,PA,None,None
2,M3,MO,D1,MO
3,M4,MO,D1,MO
4,M5,MO,D1,MO
5,M6,BO,D2,BO
6,M7,None,None,None


In [27]:
# come faccio a fare un merge e considerare solo una Citta'?
# funzione: COALESCE(...) sceglie il primo valore non nullo
q = '''
SELECT S.Matr, S.Citta, D.Citta,D.CD
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,Citta,Citta,CD
0,M1,SA,None,None
1,M2,PA,None,None
2,M3,MO,MO,D1
3,M4,MO,MO,D1
4,M5,MO,MO,D1
5,M6,BO,BO,D2
6,M7,None,None,None


In [28]:
q = '''
SELECT S.Matr, COALESCE(S.Citta, D.Citta) AS Citta, D.CD AS CodiceDocente
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,Citta,CodiceDocente
0,M1,SA,None
1,M2,PA,None
2,M3,MO,D1
3,M4,MO,D1
4,M5,MO,D1
5,M6,BO,D2
6,M7,None,None


### Self-Join

In [29]:
# Quando il join avviene tra la tabella e se stessa
# 
# Ad esempio: 
# Selezionare le coppie di studenti della stessa citta'

q = '''
SELECT S1.SNome,S2.SNome,S1.Citta
FROM S S1, S S2
WHERE S1.Citta=S2.Citta;
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome,SNome,Citta
0,Lucia Quaranta,Lucia Quaranta,SA
1,Giacomo Tedesco,Giacomo Tedesco,PA
2,Carla Longo,Carla Longo,MO
3,Carla Longo,Ugo Rossi,MO
4,Carla Longo,Valeria Neri,MO
5,Ugo Rossi,Carla Longo,MO
6,Ugo Rossi,Ugo Rossi,MO
7,Ugo Rossi,Valeria Neri,MO
8,Valeria Neri,Carla Longo,MO
9,Valeria Neri,Ugo Rossi,MO


In [32]:
# Cosi' pero' selezionaimo anche uno studente e se stesso
# Quindi modifichiamo aggiungendo:

q = '''
SELECT S1.SNome,S2.SNome,S1.Citta
FROM S S1, S S2
WHERE S1.Citta=S2.Citta
AND S1.Matr < S2.Matr;
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

#
# DOMANDA:
# perche' <> non andrebbe bene?
#

,SNome,SNome,Citta
0,Carla Longo,Ugo Rossi,MO
1,Carla Longo,Valeria Neri,MO
2,Ugo Rossi,Valeria Neri,MO


In [33]:
# Matricole degli studenti che hanno sostenuto almeno 
# uno degli esami sostenuti dallo studente di nome 'Giuseppe Verdi'

q = '''
SELECT E1.Matr
FROM S, E E1, E E2
WHERE E2.Matr = S.Matr
AND E1.CC = E2.CC
AND S.SNome='Giuseppe Verdi';
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr
0,M1
1,M2
2,M3
3,M6
4,M7


In [34]:
q = '''
SELECT E1.Matr
FROM E E1
WHERE E1.CC IN (SELECT E2.CC
				FROM E E2, S
				WHERE S.SNome='Giuseppe Verdi'
				AND S.Matr = E2.Matr);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr
0,M1
1,M2
2,M3
3,M6
4,M7


## Nested Quereis

In [35]:
# Interrogazioni innestate

# Una interrogazione viene detta innestata o nidificata se la sua condizione
# e' formulata usando il risultato di un’altra interrogazione, 
# chiamata subquery

q = '''
SELECT *
FROM S
WHERE S.Matr IN (SELECT Matr
				 FROM E);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso
0,M1,Lucia Quaranta,SA,1
1,M2,Giacomo Tedesco,PA,2
2,M3,Carla Longo,MO,1
3,M4,Ugo Rossi,MO,1
4,M6,Giuseppe Verdi,BO,1
5,M7,Maria Rossi,None,1


## ANY/IN/EXISTS
- Operatori quantificati: `<attr><op-rel>[ANY|ALL]<subquery`
- Operatore di set: `<attr> [NOT] IN <subquery>`
- Quantificatore esistenziale : `[NOT] EXISTS <subquery>`

In [50]:
#  TIPO [ANY/ALL]
# Per esprimere una condizione che e' vera quando 
# qualsiasi (ANY) opppure tutti (ALL) gli elementi
# restituiti dalla subquery fanno verificare la condizione
# i.e., condizione risulta TRUE
 
# ANY
# Nome degli studenti che hanno sostenuto l’esame del corso C1

q = '''
SELECT SNome 
FROM S 
WHERE Matr = ANY (SELECT Matr FROM E WHERE CC='C1');
'''

# SQLite non ha "ANY"
q = '''
SELECT SNome 
FROM S 
WHERE Matr IN (SELECT Matr FROM E WHERE CC='C1');
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [51]:
q = '''
SELECT SNome
FROM S natural JOIN E
WHERE E.CC = 'C1';
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [54]:
# ALL
# Studenti con anno di corso piu` basso

# SQLlite non ha "ALL"

q = '''
SELECT *
FROM S
WHERE ACorso <= ALL ( SELECT ACorso
					  FROM S);
'''

q = '''
SELECT *
FROM S
WHERE ACorso <= ( SELECT MAX(ACorso)
					  FROM S);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,Matr,SNome,Citta,ACorso
0,M1,Lucia Quaranta,SA,1
1,M2,Giacomo Tedesco,PA,2
2,M3,Carla Longo,MO,1
3,M4,Ugo Rossi,MO,1
4,M5,Valeria Neri,MO,2
5,M6,Giuseppe Verdi,BO,1
6,M7,Maria Rossi,None,1


In [55]:
# Spesso possono esserci diversi modi di esprimere la stesa query
# es: Nome degli studenti che hanno sostenuto l’esame del corso C1
# senza subquery:
q = '''
SELECT S.SNome
FROM E join S on E.Matr=S.Matr
WHERE E.CC = 'C1';
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# # oppure:
# q = '''
# SELECT S.SNome
# FROM S
# WHERE S.Matr in (SELECT E.Matr 
# 				 FROM E WHERE E.CC='C1');
# '''
# con.execute(q) 
# con.fetchdf()

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [56]:
# ALTRO ESEMPIO:
# Nome degli studenti che hanno sostenuto l’esame di un corso del docente D1
#/
q = '''
SELECT SNome
FROM E,S,C
WHERE E.Matr=S.Matr 
AND E.CC=C.CC
AND CD='D1';
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Lucia Quaranta
2,Giacomo Tedesco
3,Carla Longo
4,Ugo Rossi
5,Maria Rossi
6,Maria Rossi


In [57]:
# che puo' essere espressa anche come:
q = '''
SELECT SNome
FROM S
WHERE S.Matr IN (SELECT E.Matr 
				 FROM E 
				 WHERE E.CC IN (SELECT C.CC 
				 				FROM C 
				 				WHERE C.CD='D1'));
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Carla Longo
3,Ugo Rossi
4,Maria Rossi


In [58]:
# Provare la precedente query partendo dalla subquery piu' interna

# Fino ad ora abbiamo considerato delle subquery dove
# la query innestata era indipendente dalla query esterna

# Se vogliamo pero' selezionare:
# Per ogni citta', il nome degli studenti con anno di corso piu` alto
q = '''
SELECT S1.Citta, S1.SNome, S1.ACorso
FROM S S1
WHERE S1.ACorso >= (SELECT MAX(S2.ACorso)
					  	FROM S S2
					  	WHERE S1.Citta=S2.Citta);
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df
# S1 e' la tupla corrente

,Citta,SNome,ACorso
0,SA,Lucia Quaranta,1
1,PA,Giacomo Tedesco,2
2,MO,Valeria Neri,2
3,BO,Giuseppe Verdi,1


### EXISTS

In [59]:
# QUALIFICATORE ESISTENZIALE:

# EXISTS

# Mettendo in relazione subquery e query estenza con una clausola
# che è vera solo se per la tupla corrente il risultato della subquery
# restituisce qualcosa (i.e., non è nullo).
# Da qui: "EXISTS", cioè ESISTE qualcosa che rende TRUE quel predicato.

# Ad esempio:
# Nome degli studenti che hanno sostenuto l’esame del corso C1:

q = '''
SELECT SNome
FROM S
WHERE EXISTS (SELECT *
			  FROM E
			  WHERE E.Matr=S.Matr
			  AND E.CC = 'C1');
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [60]:
# Allo stesso modo, il negato della clausola EXISTS:

# NOT EXISTS (<subquery>)

# ha valore true se e solo se l’insieme di valori restituiti da <subquery> e' vuoto.

# Ad esempio:
# Nome degli studenti che non hanno sostenuto l’esame del corso C1
q = '''
SELECT SNome
FROM S
WHERE NOT EXISTS ( SELECT *
				   FROM E
				   WHERE E.Matr=S.Matr 
				   AND E.CC='C1' )
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# Tip:
# Se state usando EXISTS senza una subquery crrelata,
# molto probabilmente state commettendo un errore:

# Perche'?
# (hint: cosa restituisce la subquery?) 

,SNome
0,Carla Longo
1,Ugo Rossi
2,Valeria Neri
3,Giuseppe Verdi


### Riduzione di query innestate

In [63]:
# Le query innestate formulate con i seguenti operatori 
# si possono ridurre a query di join equivalenti 
# (stessa risposta per ogni possibile istanza della base di dati):
# • IN
# • ANY (con qualsiasi operatore di confronto) 
# • EXISTS con subquery correlata
# 

				   
# Nome degli studenti che hanno sostenuto l’esame del corso C1

# 1. 
q = '''
SELECT SNome 
FROM S
WHERE Matr IN ( SELECT Matr
				FROM E
				WHERE CC='C1')
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# # 2. 
# q = '''
# SELECT SNome 
# FROM S
# WHERE Matr =ANY ( SELECT Matr
# 				  FROM E
# 				  WHERE CC='C1')
# '''
# res = engine.execute(q)
# df = pd.DataFrame(res.fetchall())
# df.columns = res.keys()
# df

# 3.
q = '''
SELECT SNome 
FROM S
WHERE EXISTS ( SELECT *
			   FROM E
			   WHERE E.Matr=S.Matr
			   AND CC='C1')
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

			   
# QUERY EQUIVALENTE CON JOIN
q = '''
SELECT SNome
FROM E,S
WHERE E.Matr=S.Matr 
AND E.CC='C1'
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

,SNome
0,Lucia Quaranta
1,Giacomo Tedesco
2,Maria Rossi


In [64]:
# INVECE:
 
# Le query innestate formulate con i seguenti operatori 
# non si possono ridurre:

# • NOT IN
# • ALL (con qualsiasi operatore di confronto) 
# • NOT EXISTS con subquery correlata

# Ad esempio:
# Nome degli studenti che non hanno sostenuto l’esame del corso C1

# 1. 
q = '''
SELECT SNome 
FROM S
WHERE Matr NOT IN ( SELECT Matr
	 				FROM E
					WHERE CC='C1')
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# # 2.
# q = '''
# SELECT SNome 
# FROM S
# WHERE Matr <> ALL ( SELECT Matr
# 				  FROM E
# 				  WHERE CC='C1')
# '''
# res = engine.execute(q)
# df = pd.DataFrame(res.fetchall())
# df.columns = res.keys()
# df


# 3.
q = '''
SELECT SNome 
FROM S
WHERE NOT EXISTS ( SELECT *
			   FROM E
			   WHERE E.Matr=S.Matr
			   AND CC='C1')
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# NON SI POSSONO ESPRIMRE CON JOIN

,SNome
0,Carla Longo
1,Ugo Rossi
2,Valeria Neri
3,Giuseppe Verdi


In [65]:
# NOTARE: la differenza con la seguente query
# ERRATA (per la consegna):
q = '''
SELECT SNome
FROM E,S
WHERE E.Matr=S.Matr 
AND E.CC <> 'C1'
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# cosa restituisce?

,SNome
0,Lucia Quaranta
1,Lucia Quaranta
2,Giacomo Tedesco
3,Carla Longo
4,Carla Longo
5,Ugo Rossi
6,Giuseppe Verdi
7,Maria Rossi
8,Maria Rossi


## Funzioni di Aggregazione

In [66]:
# 		Funzioni aggregate 
# 		(column functions)

# Dato un insieme di tuple che soddisfano un predicato
# (e.g., il risultato di una query qualsiasi di quelle
# viste finora)
# le funzione di aggregazione restituiscono un risultato
# "aggregato" calcolato su queell'insieme.

# Possono essere cosi' facilmente ritrovati:
# La media dei voti di uno studete;
# Il voto massimo in un dato esame;
# Il numero di studenti che proviene da una data citta;
# ... 

q = '''
SELECT COUNT(*)
FROM S
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,COUNT(*)
0,7


In [67]:
q = '''
SELECT AVG(E.VOTO)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,AVG(E.VOTO)
0,27.416667


In [68]:
q = '''
SELECT MAX(E.VOTO)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,MAX(E.VOTO)
0,33


In [69]:
q = '''
SELECT MIN(E.VOTO)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,MIN(E.VOTO)
0,24


In [70]:
q = '''
SELECT SUM(E.VOTO)/COUNT(E.VOTO)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,SUM(E.VOTO)/COUNT(E.VOTO)
0,27


In [71]:
q = '''
SELECT DISTINCT(E.CC)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,CC
0,C1
1,C2
2,C3


In [72]:
q = '''
SELECT E.CC
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,CC
0,C1
1,C2
2,C3
3,C1
4,C2
5,C2
6,C3
7,C3
8,C2
9,C1


In [73]:
q = '''
SELECT COUNT(DISTINCT(E.CC))
FROM E
'''
# Alcuni DBMS non permettono questa sintassi
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,COUNT(DISTINCT(E.CC))
0,3


In [74]:
# Es. “Numero di studenti presenti” 
q = '''
SELECT COUNT(Matr)
FROM S
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,COUNT(Matr)
0,7


In [75]:
# Es. “Numero di studenti che hanno sostenuto almeno un esame” 
q = '''
SELECT COUNT(DISTINCT Matr)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,COUNT(DISTINCT Matr)
0,6


In [76]:
# Es. “Numero di studenti con anno di corso non nullo” 
q = '''
SELECT COUNT(ACorso)
FROM S
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,COUNT(ACorso)
0,7


In [77]:
# Es. “Numero di anni di corso di studenti presenti” 
q = '''
SELECT COUNT(DISTINCT ACorso)
FROM S
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,COUNT(DISTINCT ACorso)
0,2


In [78]:
# Es. “Numero di coppie distinte matricola-voto” 
q = '''
SELECT COUNT(DISTINCT Matr, Voto)
FROM E
'''
# Alcuni DBML non permettono questa sintassi
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


OperationalError: ignored

In [79]:
# Es. “Voto medio degli esami sostenuti dalla matricola M1”
q = '''
SELECT AVG(Voto)
FROM E 
WHERE Matr='M1'
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


# che e' equivalente a
q = '''
SELECT SUM(Voto)/COUNT(Voto) 
FROM E 
WHERE Matr='M1'
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,SUM(Voto)/COUNT(Voto)
0,29


In [80]:
# Es. “Studenti il cui anno di corso `e minore di quello massimo presente”
q = '''
SELECT *
FROM S
WHERE ACorso < ( SELECT MAX(ACorso)
				 FROM S)
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


,Matr,SNome,Citta,ACorso
0,M1,Lucia Quaranta,SA,1
1,M3,Carla Longo,MO,1
2,M4,Ugo Rossi,MO,1
3,M6,Giuseppe Verdi,BO,1
4,M7,Maria Rossi,None,1


In [81]:
# Considerate la sequente query, con funzione di aggregazione:

q = '''
SELECT Matr,MAX(Voto)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# PERCHE' E' SBAGLIATA?



,Matr,MAX(Voto)
0,M1,33
